![atc_radar](images/radar.jpg "Radar")

# TP2-ARCH-05 : Aviation : comprendre la transmission des aéronefs

## Objectifs pédagogiques

- comprendre ce qu'est un protocole *on demand* ou *broadcast*
- déduire l'architecture du système de surveillance du traffic aérien
- Construire un récepteur ADS-B avec un Raspberry Pi
- Afficher les différents aéronefs équipés ADS-B autour du gymnase

## Navigation aérienne

L'**aviation** est un système complexe : énormément de facteurs entrent en ligne de compte pour amener un aéronef d'un point A à un point B. Citons les conditions météorologiques, a présence de vent, l'état de l'aéronef, la présence d'autres aéronefs et bien évidemment le ou la pilote elle-même.

Il existe deux types de navigation autorisées aujourd'hui chacune dans un périmètre tri-dimensionnel défini. La navigation à vue ou *VFR* (pour *Visual Flight Rules*) et la navigation aux instruments ou radionavigation ou *IFR* (pour *Instruments Flight Rules*). 

## Protocole ADS-B

Le protocole ADS-B (pour *Automatic dependent surveillance-broadcast*) permet de contrôler l'espace aérien. C'est l'un des outils (parmi plusieurs redondants) utilisés pour un aéronef d'avoir une idée de sa position dans l'espace. 

L'aéronef est équipé d'un grand nombre de capteurs (vitesse, direction, altitude, position, etc..) qui sont enregistrés à intervalles réguliers. Ce sont ces informations qui sont ensuite transmises en un message ADS-B sur le mode **broadcast**, c'est-à-dire diffusion. Le grand avantage de ce système de communication est qu''il n'y a pas besoin de radar, le désavantage, l'avion doit être équipé d'un **transpondeur**, un appareil qui permet d'identifier de manière unique n'importe quel aéronef sur une bande radio donnée.

Les stations au sol peuvent ensuite capter ces messages et reconstruire la trajectoire d'un aéronef donné.

## Messages ACARS

Le système ACARS (pour *Aircraft Communication Addressing and Reporting System*)


## Utilisation des messages ADS-B et ACARS

### Elon Musk private jet

### Taylor Swift 

## Sources

- SPHAiR ([site web](https://lernprogramm.sphair.ch/fr/luftraeume/classification-de-lespace-a%C3%A9rien-suisse))
- Cartes geo.admin ICAO ([site web](https://map.geo.admin.ch/#/map?lang=de&center=2478190.86,1113421.7&z=3.407&bgLayer=ch.swisstopo.pixelkarte-farbe&topic=ech&layers=ch.bazl.luftfahrtkarten-icao&catalogNodes=687,702))
- VFR Manual Skyguide ([télécharger](https://www.skybriefing.com/documents/10156/645440/eVFR_eGEN_fr_09_2024.pdf/517b354e-2420-2c9b-de8d-4aee95c31498?t=1724226017039))
- Taylor Swift private jets tracker ([sur X (ex-Twitter)](https://x.com/SwiftJetNextDay))
- Elon Musk jet tracker ([sur X (ex-Twitter)](https://x.com/ElonJetNextDay))